In [7]:

# Importation des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import os
from google.colab import drive
from zipfile import ZipFile

# Montage Google Drive
drive.mount('/content/drive')

# Extraction du dataset
with ZipFile('/content/drive/MyDrive/AMHCD.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/AMHCD')

# Configuration des paths
dataset_path = '/content/AMHCD'
train_path = os.path.join(dataset_path, 'Train')
test_path = os.path.join(dataset_path, 'Test')

# Fonction de chargement des données
def load_amhcd_data(data_path):
    images = []
    labels = []
    class_folders = sorted(os.listdir(data_path))
    
    for class_idx, class_folder in enumerate(class_folders):
        class_path = os.path.join(data_path, class_folder)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                if img_file.endswith('.png'):
                    img_path = os.path.join(class_path, img_file)
                    # Lecture de l'image
                    img = cv2.imread(img_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (64, 64))
                    
                    images.append(img)
                    labels.append(class_idx)
    
    return np.array(images), np.array(labels)

# Chargement des données
print("Chargement des données d'entraînement...")
X_train, y_train = load_amhcd_data(train_path)
print("Chargement des données de test...")
X_test, y_test = load_amhcd_data(test_path)

print(f"Forme des données d'entraînement: {X_train.shape}")
print(f"Forme des labels d'entraînement: {y_train.shape}")
print(f"Forme des données de test: {X_test.shape}")
print(f"Forme des labels de test: {y_test.shape}")

# Normalisation des données
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encoding des labels
y_train = keras.utils.to_categorical(y_train, 33)
y_test = keras.utils.to_categorical(y_test, 33)

# Division en entraînement et validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=np.argmax(y_train, axis=1)
)

# Data Augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)

# Architecture LeNet-5 Adaptée
def create_lenet5_amhcd(input_shape=(64, 64, 3), num_classes=33):
    model = keras.Sequential([
        # Couche 1: Convolution
        layers.Conv2D(6, (5, 5), activation='relu', input_shape=input_shape, padding='same'),
        layers.AveragePooling2D((2, 2)),
        
        # Couche 2: Convolution
        layers.Conv2D(16, (5, 5), activation='relu'),
        layers.AveragePooling2D((2, 2)),
        
        # Couche 3: Convolution (ajoutée pour meilleures performances)
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.AveragePooling2D((2, 2)),
        
        # Aplatissage
        layers.Flatten(),
        
        # Couches Fully Connected
        layers.Dense(120, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(84, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Création du modèle
model = create_lenet5_amhcd()
model.summary()

# Compilation du modèle
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7
)

# Entraînement du modèle
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Évaluation du modèle
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Précision sur le jeu de test: {test_acc:.4f}")

# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Rapport de classification
print("Rapport de Classification:")
print(classification_report(y_true_classes, y_pred_classes))

# Matrice de confusion
plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de Confusion - LeNet-5 sur AMHCD')
plt.ylabel('Vraies étiquettes')
plt.xlabel('Étiquettes prédites')
plt.show()

# Courbes d'apprentissage
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy during Training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss during Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Visualisation des prédictions
class_names = [f'Classe {i}' for i in range(33)]

plt.figure(figsize=(15, 10))
for i in range(12):
    plt.subplot(3, 4, i + 1)
    idx = np.random.randint(0, len(X_test))
    plt.imshow(X_test[idx])
    true_label = class_names[y_true_classes[idx]]
    pred_label = class_names[y_pred_classes[idx]]
    confidence = np.max(y_pred[idx])
    
    color = 'green' if y_true_classes[idx] == y_pred_classes[idx] else 'red'
    plt.title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}', color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()

# Sauvegarde du modèle
model.save('lenet5_amhcd.h5')
print("Modèle sauvegardé avec succès!")

# Analyse des erreurs
errors = np.where(y_pred_classes != y_true_classes)[0]
print(f"Nombre d'erreurs: {len(errors)}/{len(X_test)} ({len(errors)/len(X_test)*100:.2f}%)")

# Visualisation des erreurs
plt.figure(figsize=(15, 10))
for i, error_idx in enumerate(errors[:12]):
    plt.subplot(3, 4, i + 1)
    plt.imshow(X_test[error_idx])
    true_label = class_names[y_true_classes[error_idx]]
    pred_label = class_names[y_pred_classes[error_idx]]
    confidence = np.max(y_pred[error_idx])
    
    plt.title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}', color='red')
    plt.axis('off')

plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'google.colab'

In [6]:
pip install opencv-python tensorflow scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [8]:


# Importation des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import os
from zipfile import ZipFile
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configuration
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

# Montage Google Drive
drive.mount('/content/drive')

# Extraction du dataset
dataset_path = '/content/AMHCD'
with ZipFile('/content/drive/MyDrive/AMHCD.zip', 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

# Fonction de chargement des données
def load_amhcd_data(data_path):
    images = []
    labels = []
    class_folders = sorted(os.listdir(data_path))
    
    for class_idx, class_folder in enumerate(class_folders):
        class_path = os.path.join(data_path, class_folder)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                if img_file.endswith('.png'):
                    img_path = os.path.join(class_path, img_file)
                    img = cv2.imread(img_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (64, 64))
                    images.append(img)
                    labels.append(class_idx)
    
    return np.array(images), np.array(labels)

# Chargement des données
print("Chargement des données d'entraînement...")
train_path = os.path.join(dataset_path, 'Train')
X_train, y_train = load_amhcd_data(train_path)

print("Chargement des données de test...")
test_path = os.path.join(dataset_path, 'Test')
X_test, y_test = load_amhcd_data(test_path)

print(f"Forme X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Forme X_test: {X_test.shape}, y_test: {y_test.shape}")

# Normalisation
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encoding
num_classes = 33
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Division entraînement/validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, 
    stratify=np.argmax(y_train, axis=1)
)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

# Architecture LeNet-5 Adaptée
def create_lenet5_amhcd(input_shape=(64, 64, 3), num_classes=33):
    model = keras.Sequential([
        # Couche 1: Convolution
        layers.Conv2D(6, (5, 5), activation='relu', 
                     input_shape=input_shape, padding='same'),
        layers.AveragePooling2D((2, 2)),
        
        # Couche 2: Convolution
        layers.Conv2D(16, (5, 5), activation='relu'),
        layers.AveragePooling2D((2, 2)),
        
        # Couche 3: Convolution additionnelle
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.AveragePooling2D((2, 2)),
        
        # Aplatissage
        layers.Flatten(),
        
        # Couches Fully Connected
        layers.Dense(120, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(84, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Création et compilation du modèle
model = create_lenet5_amhcd()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7
)

# Entraînement
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Évaluation
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Précision test: {test_acc:.4f}")

# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Métriques détaillées
print("Rapport de classification:")
print(classification_report(y_true_classes, y_pred_classes))

# Matrice de confusion
plt.figure(figsize=(15, 12))
cm = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de Confusion - LeNet-5 sur AMHCD')
plt.ylabel('Vraies étiquettes')
plt.xlabel('Étiquettes prédites')
plt.show()

# Courbes d'apprentissage
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Visualisation des prédictions
class_names = [f'Classe {i}' for i in range(33)]
plt.figure(figsize=(15, 10))
for i in range(12):
    plt.subplot(3, 4, i + 1)
    idx = np.random.randint(0, len(X_test))
    plt.imshow(X_test[idx])
    true_label = class_names[y_true_classes[idx]]
    pred_label = class_names[y_pred_classes[idx]]
    confidence = np.max(y_pred[idx])
    color = 'green' if y_true_classes[idx] == y_pred_classes[idx] else 'red'
    plt.title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}', 
              color=color, fontsize=8)
    plt.axis('off')
plt.tight_layout()
plt.show()

# Analyse des erreurs
errors = np.where(y_pred_classes != y_true_classes)[0]
print(f"Nombre d'erreurs: {len(errors)}/{len(X_test)} ({len(errors)/len(X_test)*100:.2f}%)")

# Sauvegarde
model.save('lenet5_amhcd.h5')
print("Modèle sauvegardé!")


ModuleNotFoundError: No module named 'google.colab'